# Initialize

In [ ]:
!pip install p3_data openpyxl

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
    get_varying_column_names, filter_dataframe, take_varying_columns,
    load_json_records_as_dataframe, flatten_multiindex_columns)

# Load and Clean Results

In [ ]:
%%time
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
def parse_images_per_sec(output):
    """Search for last occurance of: total images/sec: 8169.88"""
    try:
        for m in re.finditer('^total images/sec: ([.0-9]+)$', output, flags=re.MULTILINE):
            pass
        return float(m.groups()[0])
    except:
        return np.nan

In [ ]:
data_dir_suffix_info_df = pd.DataFrame([
    {'data_dir_suffix': '', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-150x', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-13x', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-png2', 'mean_bytes_per_image': 1750011440471/1281167},
]).set_index(['data_dir_suffix'])

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        r['num_gpus'] = r['np']
        r['images_per_sec'] = parse_images_per_sec(r['output'])
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['num_gpus']
        r['image_format'] = 'JPEG'
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
#r = clean_result(raw_df.iloc[-1])
#pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df = clean_df.sort_values(['utc_begin'])
clean_df['num_copies'] = clean_df['num_copies'].fillna(1.0)
clean_df.mpi = clean_df.mpi.fillna(True)
clean_df.storage_type = clean_df.storage_type.fillna('isilon')
clean_df.use_tf_layers = clean_df.use_tf_layers.fillna(True)
clean_df.nvlink = clean_df.nvlink.fillna(True)
clean_df = clean_df.join(data_dir_suffix_info_df, on=['data_dir_suffix'])
clean_df['MB_per_sec'] = clean_df['images_per_sec'] * clean_df['mean_bytes_per_image'] * 1e-6

In [ ]:
# Clean other raw results
def clean_other_result(result):
    try:
        r = result.copy()        
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['np']
        r['NVIDIA_TENSORFLOW_VERSION'] = str(r['NVIDIA_TENSORFLOW_VERSION'])
        return pd.Series(r)
    except Exception as e:
        #print('ERROR: %s: %s' % (r['test_uuid'], e))
        raise e

In [ ]:
if False:
    raw_other_results_df = pd.read_csv('other_benchmark_results.csv')
    clean_other_results_df = raw_other_results_df.apply(clean_other_result, axis=1)
    combined_df = pd.concat([clean_df, clean_other_results_df], sort=False)
else:
    combined_df = clean_df

# Explore data

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'NVIDIA_TENSORFLOW_VERSION',
 'TENSORFLOW_VERSION',
 'batch_group_size',
 'batch_size',
 'cached',
 'datasets_num_private_threads',
 'datasets_prefetch_buffer_size',
 'fp16',
 'use_tf_layers',
 'image_format',
 'model',
 'mpi',
 'np',
 'num_batches',
 'num_gpus',
 'num_hosts',
 'num_inter_threads',
 'num_intra_threads',
 'nvlink',
 'tensorflow_benchmark_git_hash',
 'test']

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'utc_begin',    
    'images_per_sec',
    'images_per_sec_per_gpu',
    'MB_per_sec',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
# Export to CSV
clean_df[cols].to_csv('results.csv')

In [ ]:
clean_df.groupby(['model']).size()

In [ ]:
clean_df.groupby(['storage_type']).size()

In [ ]:
clean_df.groupby(['isilon_node_pool_name']).size()

# NVLink vs. No NVLink

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    clean_df,
    #cached=True,
    image_format='JPEG',
    num_batches=(1000,50000),
)
len(filt_df)

In [ ]:
filt1_df = filter_dataframe(
    filt_df,
    batch_size=192,
    cached=True,
    datasets_num_private_threads=4,
#     image_format='JPEG',
#     model='resnet50',
#     np=2,
    num_batches=(500,50000),
    #num_copies=1,
#     num_gpus=1,
    num_inter_threads=40,
    num_intra_threads=1,
#     NVIDIA_TENSORFLOW_VERSION='19.09',
#     storage_type='local',
    use_tf_layers=True,
)
take_varying_columns(filt1_df[cols].sort_values(['MB_per_sec'], ascending=False)).T

In [ ]:
filt3_df = pd.concat([filt1_df])
len(filt3_df)

In [ ]:
df = filt3_df.fillna(0).groupby([
    #'NVIDIA_TENSORFLOW_VERSION',
    #'fp16',
    'model',
    'num_gpus',
#     'mpi',
#     'storage_type',
#     'cached',
#     'use_tf_layers',
    'nvlink',
#     'datasets_num_private_threads',
#     'num_inter_threads',
#     'num_intra_threads',
#     'batch_size',
]).mean()[['images_per_sec_per_gpu']]
df

In [ ]:
df2 = df.unstack([
    'num_gpus',
    #'datasets_num_private_threads',
#     'batch_size',
#     'NVIDIA_TENSORFLOW_VERSION',
#     'cached',
#     'storage_type',
#     'use_tf_layers',
    'nvlink',
])
df2

In [ ]:
df2[('images_per_sec_per_gpu',2,False)] / df2[('images_per_sec_per_gpu',2,True)]

In [ ]:
df2.plot.bar(figsize=(12,6));

# Isilon H400 vs. NVMe, no NVLink

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    clean_df,
    #cached=True,
    image_format='JPEG',
    num_batches=(1000,50000),
    nvlink=False,
)
len(filt_df)

In [ ]:
filt1_df = filter_dataframe(
    filt_df,
#     batch_size=192,
    cached=False,
#     datasets_num_private_threads=4,
#     image_format='JPEG',
#     model='resnet50',
#     np=2,
    num_batches=(500,50000),
    #num_copies=1,
#     num_gpus=1,
#     num_inter_threads=40,
#     num_intra_threads=1,
#     NVIDIA_TENSORFLOW_VERSION='19.09',
    storage_type='isilon',
    isilon_node_pool_name='h400_30tb_3.2tb-ssd_64gb',
#     use_tf_layers=True,
)
take_varying_columns(filt1_df[cols].sort_values(['MB_per_sec'], ascending=False)).T

In [ ]:
filt2_df = filter_dataframe(
    filt_df,
#     batch_size=192,
    cached=True,
#     datasets_num_private_threads=4,
#     image_format='JPEG',
#     model='resnet50',
#     np=2,
    num_batches=(500,50000),
    #num_copies=1,
#     num_gpus=1,
#     num_inter_threads=40,
#     num_intra_threads=1,
#     NVIDIA_TENSORFLOW_VERSION='19.09',
    storage_type='local',
#     use_tf_layers=True,
)
take_varying_columns(filt1_df[cols].sort_values(['MB_per_sec'], ascending=False)).T

In [ ]:
filt3_df = pd.concat([filt1_df, filt2_df])

In [ ]:
df = filt3_df.fillna(0).groupby([
    #'NVIDIA_TENSORFLOW_VERSION',
    #'fp16',
    'model',
    'num_gpus',
#     'mpi',
    'storage_type',
#     'cached',
#     'use_tf_layers',
#     'nvlink',
#     'datasets_num_private_threads',
#     'num_inter_threads',
#     'num_intra_threads',
#     'batch_size',
]).agg({'images_per_sec_per_gpu': ['mean','count']})
df

In [ ]:
df = filt3_df.fillna(0).groupby([
    #'NVIDIA_TENSORFLOW_VERSION',
    #'fp16',
    'model',
    'num_gpus',
#     'mpi',
    'storage_type',
#     'cached',
#     'use_tf_layers',
#     'nvlink',
#     'datasets_num_private_threads',
#     'num_inter_threads',
#     'num_intra_threads',
#     'batch_size',
]).agg({'images_per_sec_per_gpu': ['mean']})
df

In [ ]:
df2 = df.unstack([
#     'num_gpus',
    #'datasets_num_private_threads',
#     'batch_size',
#     'NVIDIA_TENSORFLOW_VERSION',
#     'cached',
    'storage_type',
#     'use_tf_layers',
#     'nvlink',
])
df2

In [ ]:
df2[('images_per_sec_per_gpu','mean','isilon')] / df2[('images_per_sec_per_gpu','mean','local')]

In [ ]:
df2.plot.bar(figsize=(12,6));

In [ ]:
df2.to_excel('results.xlsx')